In [1]:
import numpy as np
import pandas as pd

# Input file parameter

First we must indicated how many set are there in the TableA

In [2]:
set_total = 1

# Train file create

Open the DCADE output TableA.txt and read page file to give each node data corresponding labels (column number).

In [3]:
table_name = "./data/TableA.txt"
print("Table Opening:" + table_name + "\n")
table_a = open(table_name, "r")
output_name = "./data/train_raw.csv"
print("Generating:" + output_name + "\n")

Set_index = {}

output = open(output_name, "w")
output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tPath\tContent\tLabel\n")

line = table_a.readline()
slot = line.rstrip("\n").split("\t") # Read Table line in line and split by \t saved in slot.

while(slot[0]!="ColType"):
    '''
    Loop until we find the "ColType" in the table file to start generate training data.
    '''
    print("find coltype")
    line = table_a.readline()
    slot = line.rstrip("\n").split("\t")
col = []
line = table_a.readline() # Read first line
while(line != ""):
    slot = line.rstrip("\n").split("\t")
    page_name = slot[0]
    node_col = slot[1:] # Save each column's data in node_col
    col = []
    for i in node_col: # Split Same column data and save in list col
        tmp = i.split(" ")
        col.append(tmp)
    
    current_page = 0
    for pos in range(len(col)): # Loop all col num
        if len(col[pos]) == 1 and not col[pos][0].isnumeric(): # Find "X-X" like sets
            print("Find : "+col[pos][0])
            current_page = col[pos][0].split("-")[1]
            current_page = int(current_page)
            set_num = col[pos][0].split("-")[0]
            set_num = int(set_num)
            if set_num not in Set_index.keys():
                print("Add new set in dict col num:"+str(pos+1))
                Set_index[set_num] = pos+1 # Record it in dict
    
    page_file = pd.read_csv("./Output/"+page_name, sep='\t') # Read page file from DECADE Output file.
    
    col_name = []
    for i in page_file.columns: # Save all column name
        col_name.append(i)
    
    print("open "+ page_name)
    label_dict = {} # Define Dict of labels for each node
    for leafnode in page_file[col_name[0]]: # Loop each node in page
        for pos in range(len(col)): # Loop all col
            if str(leafnode) in col[pos]: # if node is in this column
                if leafnode not in label_dict.keys(): # and if this node not in dict save it
                    label_dict[leafnode] = pos+1 # Shift one column for every node (zero is for empty).
                #print(str(leafnode) + "\'s label: " + str(pos))
                break
    
    for set_count in range(set_total): # Loop every set and give each node a label, which is the column number of the set.
        with open("./data/Set-"+str(set_count+1)+".txt", "r") as set_file:
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            
            while(slot[0] != "ColType"):
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
            
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            while(line!=""):
                file_set_info = slot[0].split("-")
                file_set_num = set_count
                file_set_page = int(file_set_info[1])
                for node in slot[1:]:
                    if not node.isnumeric():
                        continue
                    node = int(node)
                    if file_set_page == current_page:
                        label_dict[node] = Set_index[set_count+1]
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
    #print(label_dict)
    #count = 0
    for i in label_dict.keys(): # Loop each recorded node and output as train file
        Leafnode = page_file[col_name[0]][i]
        PTypeSetid = page_file[col_name[6]][i]
        TypeSetid = page_file[col_name[7]][i]
        Contentid = page_file[col_name[8]][i].split("-")[1]
        Pathid = page_file[col_name[9]][i]
        SimSeqid = page_file[col_name[10]][i]
        Path = page_file[col_name[2]][i]
        Content = page_file[col_name[1]][i]
        output.write(str(Leafnode) + "\t")
        output.write(str(PTypeSetid) + "\t")
        output.write(str(TypeSetid) + "\t")
        output.write(str(Contentid) + "\t")
        output.write(str(Pathid) + "\t")
        output.write(str(SimSeqid) + "\t")
        output.write(str(Path) + "\t")
        output.write(str(Content) + "\t")
        output.write(str(label_dict[i]) + "\n")
    line = table_a.readline()
output.close()

Table Opening:./data/TableA.txt

Generating:./data/train_raw.csv

find coltype
find coltype
find coltype
find coltype
find coltype
find coltype
Find : 1-0
Add new set in dict col num:73
open 01.txt
Find : 1-1
open 02.txt
Find : 1-2
open 03.txt
Find : 1-3
open 04.txt
Find : 1-4
open 05.txt
Find : 1-5
open 06.txt
Find : 1-6
open 07.txt
Find : 1-7
open 08.txt
Find : 1-8
open 09.txt
Find : 1-9
open 10.txt
Find : 1-10
open 11.txt
Find : 1-11
open 12.txt
Find : 1-12
open 13.txt
Find : 1-13
open 14.txt
Find : 1-14
open 15.txt
Find : 1-15
open 16.txt
Find : 1-16
open 17.txt
Find : 1-17
open 18.txt
Find : 1-18
open 19.txt
Find : 1-19
open 20.txt
Find : 1-20
open 21.txt


In [4]:
print("Done")

Done


# Record Set data

Recorded the node index that is in a set and saved it in file.

In [5]:
if set_total > 0:
    with open("./data/Set_idx.txt", "w") as set_file:
        set_file.write(str(Set_index))
    print(Set_index)

{1: 73}


# GA file create

GA file is for testing.

In [6]:
table_name = "./data/GA.txt"
print("Table Opening:" + table_name + "\n")
table_a = open(table_name, "r")
output_name = "./data/ytest_raw.csv"
print("Generating:" + output_name + "\n")

Table Opening:./data/GA.txt

Generating:./data/ytest_raw.csv



Testing file generation is same as training file generation, but give all the nodes label 0.

In [7]:
Set_index = {}
output = open(output_name, "w")
output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tPath\tContent\tLabel\n")
line = table_a.readline()
slot = line.rstrip("\n").split("\t")
while(slot[0]!="ColType"):
    print("find coltype")
    line = table_a.readline()
    slot = line.rstrip("\n").split("\t")
line = table_a.readline() # Read
while(line != ""):
    slot = line.rstrip("\n").split("\t")
    page_name = slot[0]
    if slot[0]=="":
        break
    page_file = pd.read_csv("./Output/"+page_name, sep='\t')
    col_name = []
    for i in page_file.columns:
        col_name.append(i)
    print("open "+ page_name)
    label_dict = {}
    for node in range(len(page_file[col_name[0]])):
        Leafnode = page_file[col_name[0]][node]
        PTypeSetid = page_file[col_name[6]][node]
        TypeSetid = page_file[col_name[7]][node]
        Contentid = page_file[col_name[8]][node].split("-")[1]
        Pathid = page_file[col_name[9]][node]
        SimSeqid = page_file[col_name[10]][node]
        Path = page_file[col_name[2]][node]
        Content = page_file[col_name[1]][node]
        output.write(str(Leafnode) + "\t")
        output.write(str(PTypeSetid) + "\t")
        output.write(str(TypeSetid) + "\t")
        output.write(str(Contentid) + "\t")
        output.write(str(Pathid) + "\t")
        output.write(str(SimSeqid) + "\t")
        output.write(str(Path) + "\t")
        output.write(str(Content) + "\t")
        output.write(str(0) + "\n")
    line = table_a.readline() # Read
output.close()

open 01.txt
open 02.txt
open 03.txt
open 04.txt
open 05.txt
open 06.txt
open 07.txt
open 08.txt
open 09.txt
open 10.txt
open 11.txt
open 12.txt
open 13.txt
open 14.txt
open 15.txt
open 16.txt
open 17.txt
open 18.txt
open 19.txt
open 20.txt
open 21.txt


In [8]:
print("Done!")

Done!
